In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=70175611cbd43cab4564f0c86529987ec90a55d7e1367141651a532a51976420
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 2706.82 ,random_state=0 , l1_ratio=0.997, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_10_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=846490, Mon Apr 13 13:55:16 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.037630676035470e+08 1.0e+00 2.57e-01  3e-01  3e-01 0:00.0
    2     30 1.037618370202356e+08 1.0e+00 2.51e-01  2e-01  3e-01 0:00.0
    3     45 1.037616398266883e+08 1.1e+00 2.48e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.03769021e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.03776915e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.297756325425783e+08 1.5e+00 4.97e-02  5e-02  5e-02 0:01.9
  200   3000 -1.431824232990540e+08 1.5e+00 2.88e-03  3e-03  3e-03 0:03.9
  300   4500 -1.432351166040957e+08 1.5e+00 1.90e-04  2e-04  2e-04 0:05.9
  400   6000 -1.432353829258144e+08 1.5e+00 1.33e-05  1e-05  1e-05 0:07.8
  500   7500 -1.432353843234666e+08 1.6e+00 7.16e-07  6e-07  6e-07 0:09.8
  600   9000 -1.432353843277915e+08 1.6e+00 4.21e-08  3e-08  4e-08 0:11.7
  700  10500 -1.432353843278006e+08 1.7e+00 3.77e-09  3e-09  3e-09 0:13.6
  800  12000 -1.432353843278007e+08 2.0e+00 1.61e-09  1e-09  1e-09 0:15.6
  900  13500 -1.432353843278006e+08 2.1e+00 1.45e-09  1e-09  1e-09 0:17.5
 1000  15000 -1.432353843278007e+08 2.3e+00 8.09e-10  6e-10  7e-10 0:19.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.432353843278006e+08 2.5e+00 7.19e-10  5e-10  6e-10 0:21.4
 1200  18000 -1.432353843278006e+08 2.6e+00 5.92e-10  4e-10  5e-10 0:23.4
 1300  19500 -1.432353843278007e+08 2.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [96867203.60768202] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 -6.076133530032777e+11 1.6e+00 1.82e+02  2e+02  2e+02 0:01.9
  105   1575 -1.236562116142572e+12 1.6e+00 2.72e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 13:56:16 2020)
final/bestever f-value = -1.152834e+12 -1.236562e+12
incumbent solution: [  251.58029703   431.50302807 -1144.61752437   321.0039954
   723.5245552   -590.52911011     4.45424433  -455.54988197 ...]
std deviations: [265.47518003 262.24847235 263.70011167 261.78455784 266.30084475
 263.75962414 264.62797682 259.40397911 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=867647, Mon Apr 13 13:56:16 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.037575616593032e+08 1.0e+00 2.56e-01  3e-01  3e-01 0:00.0
    2     30 1.037266325743482e+08 1.0e+00 2.49e-01  2e-01  3e-01 0:00.0
    3     45 1.037196378126260e+08 1.1e+00 2.45e-01  2e-01  2e-01 0:00.1


...

  100   1500 -3.571460858810317e+11 1.5e+00 1.29e+02  1e+02  1e+02 0:01.9
  109   1635 -1.406153989810676e+12 1.6e+00 2.63e+02  2e+02  3e+02 0:02.1
termination on tolfacupx=1000.0 (Mon Apr 13 14:27:46 2020)
final/bestever f-value = -1.316162e+12 -1.406154e+12
incumbent solution: [ 663.54360744  552.77617705 -265.63052268   44.53727139  666.0358104
 -172.85419181  459.43949857  256.53146895 ...]
std deviations: [258.67283474 255.03455764 255.16283029 253.29280838 260.7566097
 254.22983446 249.0930498  253.29326377 ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=969092, Mon Apr 13 14:27:46 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.037649763412622e+08 1.0e+00 2.57e-01  3e-01  3e-01 0:00.0
    2     30 1.037625150228454e+08 1.0e+00 2.50e-01  2e-01  3e-01 0:00.0
    3     45 1.037619619954368e+08 1.1e+00 2.49e-01  2e-01  3e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.03828446e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.03765841e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.210006714262276e+08 1.5e+00 4.74e-02  4e-02  5e-02 0:02.0
  200   3000 -1.431514977758332e+08 1.6e+00 3.20e-03  3e-03  3e-03 0:04.0
  300   4500 -1.432351804187338e+08 1.6e+00 1.73e-04  1e-04  2e-04 0:06.0
  400   6000 -1.432353833000565e+08 1.6e+00 1.06e-05  9e-06  1e-05 0:07.9
  500   7500 -1.432353843238776e+08 1.7e+00 7.77e-07  6e-07  7e-07 0:09.9
  600   9000 -1.432353843277839e+08 1.7e+00 5.77e-08  4e-08  5e-08 0:11.9
  700  10500 -1.432353843278006e+08 1.7e+00 4.32e-09  3e-09  4e-09 0:13.9
  800  12000 -1.432353843278006e+08 1.9e+00 1.91e-09  1e-09  2e-09 0:15.8
  900  13500 -1.432353843278007e+08 2.1e+00 1.51e-09  1e-09  1e-09 0:17.8
 1000  15000 -1.432353843278008e+08 2.3e+00 1.12e-09  8e-10  9e-10 0:19.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.432353843278006e+08 2.5e+00 1.08e-09  7e-10  9e-10 0:21.8
 1200  18000 -1.432353843278006e+08 2.8e+00 8.76e-10  6e-10  7e-10 0:23.7
 1300  19500 -1.432353843278007e+08 2.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [96867203.60768202] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.53e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.53e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 -1.897247314783722e+11 1.4e+00 1.07e+02  1e+02  1e+02 0:01.9
  113   1695 -1.221114437480670e+12 1.5e+00 2.74e+02  3e+02  3e+02 0:02.2
termination on tolfacupx=1000.0 (Mon Apr 13 14:28:57 2020)
final/bestever f-value = -1.146009e+12 -1.221114e+12
incumbent solution: [ -75.64459771  -66.67467069  219.75680642 -192.11403804  -23.96213073
  845.98658916 -464.17048868  306.18851065 ...]
std deviations: [262.57819111 265.03666487 272.35260303 264.53423908 268.2027434
 267.49113521 261.74473339 262.44648707 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F10_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
KNN_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:50])
    SVM_Fun [i] = F10(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F10(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([8553992.66490083]), count=array([1])) 1121728.745510857
SVM
ModeResult(mode=array([5413118.16323401]), count=array([1])) 1287700.9405228028
ELN
ModeResult(mode=array([5.61710041e+11]), count=array([1])) 710739228899.9099
